## RGA + TF-IDF
### RGA + TF‑IDF 协同流程
1. 生成查询扩写：LLM 产生 2‑5 个不同角度的关键词表达。
2. 逐条执行 TF‑IDF 检索 得到候选文档。(① 统计词频‑逆文档频率 TF·IDF
② 把文档与查询表示成稀疏向量
③ 用余弦相似度打分)
3. 融合去重：合并多条检索结果，可再按 TF‑IDF 分数或简单规则排序。

In [9]:
import os
from typing import List

from langchain_community.embeddings import DashScopeEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI

# ======= 配置 =======
# ----------- 配置 -----------
DASH_API_KEY = "sk-0b8d48b85f1742829ef3032133375d3e"
EMBED_MODEL = "text-embedding-v2"
LLM_MODEL = "qwen2.5-72b-instruct"

embedding = DashScopeEmbeddings(model=EMBED_MODEL, dashscope_api_key=DASH_API_KEY)
llm = ChatOpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
                 api_key=DASH_API_KEY,
                 model=LLM_MODEL, temperature=0.7)
TOP_K = 5
EXPANSION_N = 3

# ======= 示例文档 =======
docs = [
    Document(page_content="打印机应在通风环境下使用，防止过热。", metadata={"source": "safety"}),
    Document(page_content="安装打印驱动程序前，请确保系统兼容。", metadata={"source": "setup"}),
    Document(page_content="打印机使用时请远离水源，避免短路。", metadata={"source": "safety"}),
    Document(page_content="节能打印模式可降低耗电量。", metadata={"source": "efficiency"}),
    Document(page_content="长期不使用打印机应拔掉电源。", metadata={"source": "safety"}),
]

# ======= 构建 TF-IDF 索引 =======
corpus = [d.page_content for d in docs]
vectorizer = TfidfVectorizer()
doc_vecs = vectorizer.fit_transform(corpus)

# ======= 查询扩写器 (RGA) =======
def build_query_expander():
    prompt = PromptTemplate.from_template(
        """你是一个查询增强助手，请基于以下用户查询，生成 {n} 条不同表达的改写查询，每条一行：
        用户查询：{query}
        改写查询："""
    )
    return LLMChain(llm=llm, prompt=prompt)

# ======= 检索函数 =======
def rga_tfidf_search(query: str, expander: LLMChain):
    # (1) 查询扩写
    expanded = expander.run({"query": query, "n": EXPANSION_N})
    queries = [query] + [l.strip(" 123456.-") for l in expanded.split("\n") if l.strip()]
    print("🔍 扩展查询：", queries)

    # (2) TF-IDF 检索 + 相似度打分
    results = []
    seen = set()
    for q in queries:
        q_vec = vectorizer.transform([q])
        scores = cosine_similarity(q_vec, doc_vecs)[0]
        sorted_indices = scores.argsort()[::-1]

        for idx in sorted_indices[:TOP_K]:
            doc = docs[idx]
            score = scores[idx]
            if doc.page_content not in seen:
                results.append((doc, score))
                seen.add(doc.page_content)

    return sorted(results, key=lambda x: x[1], reverse=True)[:TOP_K]

# ======= 主程序 =======
if __name__ == "__main__":
    query = "节能打印模式可降低耗电量？"
    expander = build_query_expander()
    results = rga_tfidf_search(query, expander)

    print(f"\n🎯 Top {len(results)} 文档：")
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n[{i}] (score={score:.4f})\n{doc.page_content}")


🔍 扩展查询： ['节能打印模式可降低耗电量？', '节能打印模式能否减少电力消耗？', '使用节能打印模式可以节省电吗？', '开启节能打印模式对降低功耗有帮助吗？']

🎯 Top 5 文档：

[1] (score=1.0000)
节能打印模式可降低耗电量。

[2] (score=0.0000)
长期不使用打印机应拔掉电源。

[3] (score=0.0000)
打印机使用时请远离水源，避免短路。

[4] (score=0.0000)
安装打印驱动程序前，请确保系统兼容。

[5] (score=0.0000)
打印机应在通风环境下使用，防止过热。


## RGA  BM25

In [12]:
import os, re
from typing import List, Tuple, Dict

from langchain_community.embeddings import DashScopeEmbeddings
from rank_bm25 import BM25Okapi
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
# ----------- 配置 -----------
DASH_API_KEY = "sk-0b8d48b85f1742829ef3032133375d3e"
EMBED_MODEL = "text-embedding-v2"
LLM_MODEL = "qwen2.5-72b-instruct"

embedding = DashScopeEmbeddings(model=EMBED_MODEL, dashscope_api_key=DASH_API_KEY)
LLM = ChatOpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
                 api_key=DASH_API_KEY,
                 model=LLM_MODEL, temperature=0.7)

EXPANSION_N = 3          # RGA 生成查询数
TOP_K = 5                # 每条子查询 BM25 取前 k

# =========== ② 微型语料 ===========
RAW_DOCS = [
    "打印机应在通风环境下使用，防止设备过热并降低火灾风险。",
    "维修打印机前必须断电，以避免触电和机械伤害。",
    "节能模式可将打印机能耗降低最多 30%。",
    "使用原装耗材可提高打印质量并延长设备寿命。",
    "打印前请检查电缆连接是否牢固，避免短路隐患。"
]
docs: List[Document] = [
    Document(page_content=txt, metadata={"id": f"d{i}"}) for i, txt in enumerate(RAW_DOCS)
]

# =========== ③ 极简中文分词 ===========
def tokenize(text: str) -> List[str]:
    """正则切词，可替换为 jieba / pkuseg 以获得更好分词效果"""
    return re.findall(r"\w+", text.lower(), flags=re.U)

# 构建 BM25 索引
corpus_tokens = [tokenize(d.page_content) for d in docs]
bm25 = BM25Okapi(corpus_tokens)

# =========== ④ 查询扩写链（RGA） ===========
expander = LLMChain(
    llm=LLM,
    prompt=PromptTemplate.from_template(
        "请为以下查询生成 {n} 条不同表达，每行一个：\n查询：{query}\n改写："
    )
)

# =========== ⑤ RGA + BM25 检索 ===========
def rga_bm25_search(query: str) -> List[Tuple[Document, float]]:
    # 1. LLM 生成改写
    exp = expander.run({"query": query, "n": EXPANSION_N})
    queries = [query] + [q.strip(" 123456.-") for q in exp.split("\n") if q.strip()]
    print("🔍 扩写查询:", queries)

    # 2. 每条子查询 BM25 召回
    scored: Dict[str, float] = {}   # doc_id -> score
    for q in queries:
        q_tok = tokenize(q)
        scores = bm25.get_scores(q_tok)
        top_idx = scores.argsort()[-TOP_K:][::-1]     # 最高 k
        for idx in top_idx:
            doc_id = docs[idx].metadata["id"]
            scored[doc_id] = max(scored.get(doc_id, 0), scores[idx])  # 取最大分

    # 3. 降序排序并返回
    results = sorted([(docs[int(d[1:])], s) for d, s in scored.items()],
                     key=lambda x: x[1], reverse=True)
    return results[:TOP_K]

# =========== ⑥ Demo 运行 ===========
if __name__ == "__main__":
    user_query = "打印机能耗降低最多 30%。"
    for i, (doc, score) in enumerate(rga_bm25_search(user_query), 1):
        print(f"\n[{i}] BM25={score:.2f} | {doc.page_content}")


🔍 扩写查询: ['打印机能耗降低最多 30%。', '打印机节能最高达 30%。', '打印机的能源消耗减少不超过 30%。', '打印机能效提升，最大节能比例为 30%。']

[1] BM25=1.05 | 节能模式可将打印机能耗降低最多 30%。

[2] BM25=0.00 | 打印前请检查电缆连接是否牢固，避免短路隐患。

[3] BM25=0.00 | 使用原装耗材可提高打印质量并延长设备寿命。

[4] BM25=0.00 | 维修打印机前必须断电，以避免触电和机械伤害。

[5] BM25=0.00 | 打印机应在通风环境下使用，防止设备过热并降低火灾风险。


## RGA + SPLADE
## https://github.com/naver/splade
